In [41]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [42]:
olympic_history = "./athlete_events.csv"
o_history_df = pd.read_csv(olympic_history)
o_history_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [58]:
noc_regions = "./noc_regions.csv"
regions_df = pd.read_csv(noc_regions)
regions_df.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


In [59]:
reg_cols = ["NOC", "region"]
reg_cat_df = regions_df[reg_cols]
reg_cat_df.head()

,NOC,region
0,AFG,Afghanistan
1,AHO,Curacao
2,ALB,Albania
3,ALG,Algeria
4,AND,Andorra


In [44]:
cols = ["Team","Games", "Season", "City", "Medal", "Sport", "Event"]
olympic_cat_df = o_history_df[cols]
olympic_cat_df.head()


,Team,Games,Season,City,Medal,Sport,Event
0,China,1992 Summer,Summer,Barcelona,NaN,Basketball,Basketball Men's Basketball
1,China,2012 Summer,Summer,London,NaN,Judo,Judo Men's Extra-Lightweight
2,Denmark,1920 Summer,Summer,Antwerpen,NaN,Football,Football Men's Football
3,Denmark/Sweden,1900 Summer,Summer,Paris,Gold,Tug-Of-War,Tug-Of-War Men's Tug-Of-War
4,Netherlands,1988 Winter,Winter,Calgary,NaN,Speed Skating,Speed Skating Women's 500 metres


In [45]:
olympic_cat_df.count()

Team      271116
Games     271116
Season    271116
City      271116
Medal      39783
Sport     271116
Event     271116
dtype: int64

In [46]:
no_null_olympic_cat_df = olympic_cat_df.dropna(how='any')
no_null_olympic_cat_df.head()

,Team,Games,Season,City,Medal,Sport,Event
3,Denmark/Sweden,1900 Summer,Summer,Paris,Gold,Tug-Of-War,Tug-Of-War Men's Tug-Of-War
37,Finland,1920 Summer,Summer,Antwerpen,Bronze,Swimming,Swimming Men's 200 metres Breaststroke
38,Finland,1920 Summer,Summer,Antwerpen,Bronze,Swimming,Swimming Men's 400 metres Breaststroke
40,Finland,2014 Winter,Winter,Sochi,Bronze,Ice Hockey,Ice Hockey Men's Ice Hockey
41,Finland,1948 Summer,Summer,London,Bronze,Gymnastics,Gymnastics Men's Individual All-Around


In [54]:
no_null_olympic_cat_df["Medal"].value_counts()

Gold      13372
Bronze    13295
Silver    13116
Name: Medal, dtype: int64

In [56]:
new_data_df = no_null_olympic_cat_df
new_data_df.head()

,Team,Games,Season,City,Medal,Sport,Event
3,Denmark/Sweden,1900 Summer,Summer,Paris,Gold,Tug-Of-War,Tug-Of-War Men's Tug-Of-War
37,Finland,1920 Summer,Summer,Antwerpen,Bronze,Swimming,Swimming Men's 200 metres Breaststroke
38,Finland,1920 Summer,Summer,Antwerpen,Bronze,Swimming,Swimming Men's 400 metres Breaststroke
40,Finland,2014 Winter,Winter,Sochi,Bronze,Ice Hockey,Ice Hockey Men's Ice Hockey
41,Finland,1948 Summer,Summer,London,Bronze,Gymnastics,Gymnastics Men's Individual All-Around


In [60]:
new_data_df["Sport"].unique()

array(['Tug-Of-War', 'Swimming', 'Ice Hockey', 'Gymnastics',
       'Alpine Skiing', 'Handball', 'Hockey', 'Rowing', 'Football',
       'Speed Skating', 'Sailing', 'Cycling', 'Fencing', 'Taekwondo',
       'Athletics', 'Canoeing', 'Water Polo', 'Wrestling',
       'Modern Pentathlon', 'Figure Skating', 'Golf', 'Softball',
       'Boxing', 'Basketball', 'Nordic Combined', 'Diving', 'Baseball',
       'Volleyball', 'Cross Country Skiing', 'Bobsleigh', 'Curling',
       'Shooting', 'Judo', 'Equestrianism', 'Tennis', 'Rugby Sevens',
       'Rhythmic Gymnastics', 'Weightlifting', 'Badminton',
       'Beach Volleyball', 'Ski Jumping', 'Rugby',
       'Short Track Speed Skating', 'Biathlon', 'Lacrosse',
       'Synchronized Swimming', 'Archery', 'Freestyle Skiing',
       'Triathlon', 'Polo', 'Luge', 'Table Tennis', 'Snowboarding',
       'Cricket', 'Skeleton', 'Racquets', 'Military Ski Patrol',
       'Croquet', 'Art Competitions', 'Roque', 'Alpinism', 'Trampolining',
       'Basque Pelota',

In [64]:
swimming_data_df = new_data_df.loc[new_data_df["Sport"] == "Swimming", :]
swimming_data_df.head()

,Team,Games,Season,City,Medal,Sport,Event
37,Finland,1920 Summer,Summer,Antwerpen,Bronze,Swimming,Swimming Men's 200 metres Breaststroke
38,Finland,1920 Summer,Summer,Antwerpen,Bronze,Swimming,Swimming Men's 400 metres Breaststroke
210,Hungary,1936 Summer,Summer,Berlin,Bronze,Swimming,Swimming Men's 4 x 200 metres Freestyle Relay
476,Canada,1984 Summer,Summer,Los Angeles,Bronze,Swimming,Swimming Women's 4 x 100 metres Medley Relay
740,South Africa,1956 Summer,Summer,Melbourne,Bronze,Swimming,Swimming Women's 4 x 100 metres Freestyle Relay


In [65]:
speed_skating_df = new_data_df.loc[new_data_df["Sport"] == "Speed Skating"]
speed_skating_df.head()

,Team,Games,Season,City,Medal,Sport,Event
110,Norway,1952 Winter,Winter,Oslo,Bronze,Speed Skating,"Speed Skating Men's 1,500 metres"
113,Norway,1960 Winter,Winter,Squaw Valley,Gold,Speed Skating,"Speed Skating Men's 1,500 metres"
974,Russia,2006 Winter,Winter,Torino,Bronze,Speed Skating,Speed Skating Women's Team Pursuit (6 laps)
4130,East Germany,1980 Winter,Winter,Lake Placid,Bronze,Speed Skating,"Speed Skating Women's 1,000 metres"
6674,Norway,1952 Winter,Winter,Oslo,Gold,Speed Skating,"Speed Skating Men's 1,500 metres"


In [77]:
sport_merge = pd.merge(speed_skating_df, swimming_data_df,
                       on="Sport", how="outer")
sport_merge

,Team_x,Games_x,Season_x,City_x,Medal_x,Sport,Event_x,Team_y,Games_y,Season_y,City_y,Medal_y,Event_y
0,Norway,1952 Winter,Winter,Oslo,Bronze,Speed Skating,"Speed Skating Men's 1,500 metres",NaN,NaN,NaN,NaN,NaN,NaN
1,Norway,1960 Winter,Winter,Squaw Valley,Gold,Speed Skating,"Speed Skating Men's 1,500 metres",NaN,NaN,NaN,NaN,NaN,NaN
2,Russia,2006 Winter,Winter,Torino,Bronze,Speed Skating,Speed Skating Women's Team Pursuit (6 laps),NaN,NaN,NaN,NaN,NaN,NaN
3,East Germany,1980 Winter,Winter,Lake Placid,Bronze,Speed Skating,"Speed Skating Women's 1,000 metres",NaN,NaN,NaN,NaN,NaN,NaN
4,Norway,1952 Winter,Winter,Oslo,Gold,Speed Skating,"Speed Skating Men's 1,500 metres",NaN,NaN,NaN,NaN,NaN,NaN
5,Norway,1952 Winter,Winter,Oslo,Gold,Speed Skating,"Speed Skating Men's 5,000 metres",NaN,NaN,NaN,NaN,NaN,NaN
6,Norway,1952 Winter,Winter,Oslo,Gold,Speed Skating,"Speed Skating Men's 10,000 metres",NaN,NaN,NaN,NaN,NaN,NaN
7,Norway,1980 Winter,Winter,Lake Placid,Bronze,Speed Skating,"Speed Skating Men's 1,500 metres",NaN,NaN,NaN,NaN,NaN,NaN
8,Italy,2006 Winter,Winter,Torino,Gold,Speed Skating,Speed Skating Men's Team Pursuit (8 laps),NaN,NaN,NaN,NaN,NaN,NaN
9,Germany,2006 Winter,Winter,Torino,Gold,Speed Skating,Speed Skating Women's Team Pursuit (6 laps),NaN,NaN,NaN,NaN,NaN,NaN
